# Fragmenstein
Fragmenstein is a position-based fragment-merging python3 tool.

<img src="https://github.com/matteoferla/Fragmenstein/raw/master/images/overview.png" width="800" alt="logo">

In its merging/linking operation, under the coordination of the class Victor,
the class Monster finds spatially overlapping atoms and stitches them together (with RDKit),
then the class Igor reanimates (minimises in PyRosetta) them within the protein site restraining the atoms to original positions.
As this compound may not be purchasable, one can use the placement operation to
make a stitched-together molecule based a template.

# Further details

## Details

This notebook does sevaral operations as examples.

* It optionally minimises the template structure, 
* optionally extracts the hits from provided PDB structures.
* It combines combinatorially the provided hits
* It then searches for the most similar molecules to the user chosen molecule in the Enamine Real database (via the API of John Irwin's SmallWorld server)
* places them.

NB Whereas Fragmenstein can deal with covalent ligands
and can interconvert a few cysteine reactive warheads
this notebook does not do any of that due to corner case mayhem. The demo data (MPro from the [Covid Moonshot](https://covid.postera.ai/covid) available from [Fragalysis](https://fragalysis.diamond.ac.uk/)) will use covalent residues.

Fragmenstein can _partially_ work without PyRosetta, but this notebook does not work that way.

This notebook from https://github.com/matteoferla/Fragmenstein.

It is meant to be opened in Colabs via [https://colab.research.google.com/github/matteoferla/Fragmenstein/blob/master/colab_fragmenstein.ipynb](https://colab.research.google.com/github/matteoferla/Fragmenstein/blob/master/colab_fragmenstein.ipynb)

See also:

* https://github.com/matteoferla/Fragmenstein
* https://fragmenstein.readthedocs.io
* https://github.com/matteoferla/Python_SmallWorld_API
* https://github.com/matteoferla/pyrosetta_help

# Installation and initialisation

In [ ]:
#@title Installation
#@markdown Press the play button on the top right hand side of this cell
#@markdown once you have checked the settings.
#@markdown You will be notified that this notebook is not from Google, that is normal.

#@markdown Send error messages to errors.matteoferla.com for logging?
#@markdown See [notebook-error-reporter repo for more](https://github.com/matteoferla/notebook-error-reporter)
report_errors = False #@param {type:"boolean"}
if report_errors:
    !pip install notebook-error-reporter
    from notebook_error_reporter import ErrorServer

    es = ErrorServer(url='https://errors.matteoferla.com', notebook='fragmenstein')
    es.enable()


!pip install -q rdkit rdkit-to-params biopython pyrosetta-help jsme_notebook
!pip install -q --upgrade plotly


import os
from importlib import reload
import pyrosetta_help as ph
from typing import *

# Muppet-proofing: are we in colab?
assert ph.get_shell_mode() == 'colab', 'This is a colab notebook, if running in Jupyter notebook, the installation is different'

# Colab still runs on 3.7
# hack to enable the backport:
import sys
if sys.version_info.major != 3 or sys.version_info.minor < 8:
    !pip install -q singledispatchmethod
    import functools
    from singledispatchmethod import singledispatchmethod  # noqa it's okay, PyCharm, I am not a technoluddite
    functools.singledispatchmethod = singledispatchmethod
    !pip install -q typing_extensions
    import typing_extensions
    import typing
    for key, fun in typing_extensions.__dict__.items():
      if key not in typing.__dict__:
        setattr(typing, key, fun)

# ============================================================================
#@markdown ### Use Google Drive
#@markdown Optionally store your results in your google drive.
#@markdown If `use_drive` is True, you will be prompted to give permission to use Google Drive
#@markdown (you may be prompted to follow a link and possibly authenticate and then copy a code into a box)
#@markdown —**_always_ remember to check strangers' code against data theft**:
#@markdown e.g. search and look for all instances of `http`, `requests` and `post` in the code, and
#@markdown make sure the creator is not typosquatting as someone else (e.g. username `Coogle`).
use_drive = True  #@param {type:"boolean"}
if use_drive:
    ph.mount_google_drive(use_drive)

# ============================================================================
#@markdown ### PyRosetta installation
#@markdown This will install PyRosetta in this Colab notebook (~2–15 minutes depending on time of day),
#@markdown but you will require a [PyRosetta licence](https://www.pyrosetta.org/home/licensing-pyrosetta)
#@markdown (free for academics).
#@markdown to speed things up _next_ time you can download a release into your Google Drive.
#@markdown Use Google Drive for PyRosetta:

#@markdown &#x1F44D; maybe faster next time

#@markdown &#x1F44E; occupies some 10 GB, so you'll need to be on the 100 GB plan of Google Drive (it's one pound a month).

download_to_drive = False #@param {type:"boolean"}
download_path = '.' #@param {type:"string"}
#@markdown If this is the next-time, `download_to_drive` and the credentials below will be ignored if
#@markdown there's a release in `download_path`.

#@markdown The following is not the real username and password. However, the format is similar.
username = 'boltzmann'  #@param {type:"string"}
username.strip().lower()
password = 'constant'  #@param {type:"string"}
#@markdown If yours are not the academic credentials
#@markdown disable this:
hash_comparison_required = True #@param {type:"boolean"}
#@markdown &#x1f4a1; THIS FLAG IS NOT PREVENTING YOU FROM USING PLAIN ROSETTA CREDENTIALS
#@markdown AS THE CUSTOM ERROR SAYS! **REGULAR ROSETTA CREDENTIALS DO NOT WORK FOR PYROSETTA.**

if download_to_drive and not use_drive:
    raise ValueError('You said False to `use_drive` and True to `download_to_drive`? Very funny.')
elif download_to_drive:
    ph.download_pyrosetta(username=username,
                          password=password,
                          path=download_path,
                          hash_comparison_required=hash_comparison_required)
else:
    pass

ph.install_pyrosetta(username=username,
                     password=password,
                     path=download_path,
                     hash_comparison_required=hash_comparison_required)
reload(ph)

# ??????? NOTE ??????????????????????????????????????????????????????????????????????
# ? Note to code spies
# ? this is a convoluted way to install pyrosetta via pyrosetta_help, due to options.
# ? the quicker way is:
# ?
# ? >>> pip install pyrosetta-help
# ? >>> install_pyrosetta -u xxx -p xxx
# ??????????????????????????????????????????????????????????????????????????????????

# disable as appropriate:
!pip install py3Dmol
# or:
!pip install nglview
from google.colab import output  # noqa (It's a colaboratory specific repo)
output.enable_custom_widget_manager()



# refresh imports
import site
site.main()

# ================================================================
# ================================================================
# Fragmenstein
# custom for this notebook
!pip install smallworld_api fragmenstein
# Hey, future me, debugging are we? try:
# os.system('pip install git+https://github.com/matteoferla/Fragmenstein.git@dev-teo3')

# make folders in _path
working_folder = 'fragmenstein_data'
input_folder = 'input'
output_folder = 'output'
if not os.path.exists(working_folder):
    os.mkdir(working_folder)
os.chdir('fragmenstein_data')
for folder in (input_folder, output_folder):
    if not os.path.exists(folder):
        os.mkdir(folder)

# ================================================================
# 3D viewer

from typing import *
from rdkit import Chem
import py3Dmol


def stylize(representation: str, color: str) -> Dict[str, Dict[str, Dict[str, str]]]:
    if 'carbon' in color.lower():
        return dict(style={representation: {'colorscheme': color}})
    else:
        return dict(style={representation: {'color': color}})


def make_3Dview(template_pdbblock, colormols: Dict[str, List[Chem.Mol]]) -> py3Dmol.view:
    """
    colormols is a diction of color/colorscheme to list of mols.
    """
    view = py3Dmol.view(js="https://3dmol.org/build/3Dmol.js")
    view.addModel(template_pdbblock, "pdb", stylize('cartoon', 'gainsboro'))
    view.setStyle(dict(hetflag=True), stylize('stick', 'whiteCarbon'))
    for color, mols in colormols.items():
        for mol in mols:
            view.addModel(Chem.MolToMolBlock(mol), "mol", stylize('stick', color))
    view.zoomTo(dict(hetflag=True))
    return view


# ================================================================
# mol grid

from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from IPython.display import display


def display_mols(mols: Sequence[Chem.Mol],
                 molsPerRow=5,
                 subImgSize=(150,150),
                 useSVG=True):
    """
    Rudimentary wrapper for calling ``display(Draw.MolsToGridImage``
    """
    flattos = [AllChem.RemoveHs(mol) for mol in mols if isinstance(mol, Chem.Mol)]
    for mol in flattos:
        AllChem.Compute2DCoords(mol)
    display(Draw.MolsToGridImage(flattos,
                         legends=[mol.GetProp('_Name') if mol.HasProp('_Name') else '-' for mol in mols],
                         subImgSize=subImgSize, useSVG=useSVG,
                         molsPerRow=molsPerRow))


# ================================================================

In [ ]:
#@title Start PyRosetta
#@markdown Leave alone and just run it. 
#@markdown Only one that you might want to change is `ignore_waters`
#@markdown as merging a ligand with its watershell may be a reasonable thing to do
#@markdown in extreme circumstances —like not even Chuck Norris could find a followup.
import pyrosetta, logging
import pyrosetta_help as ph

#@markdown Do not optimise hydrogen on loading:
no_optH = False  #@param {type:"boolean"}
#@markdown Ignore (True) or raise error (False) if novel residue (e.g. ligand) —  **don't tick this**.
ignore_unrecognized_res = False  #@param {type:"boolean"}
#@markdown Use autogenerated PDB residues are often weird (bad geometry, wrong match, protonated etc.): —best do it properly and parameterise it, so **don't tick this**.
load_PDB_components = False  #@param {type:"boolean"}
#@markdown Ignore all waters:
ignore_waters = True  #@param {type:"boolean"}

extra_options = ph.make_option_string(no_optH=no_optH,
                                      ex1=None,
                                      ex2=None,
                                      mute='all',
                                      ignore_unrecognized_res=ignore_unrecognized_res,
                                      load_PDB_components=load_PDB_components,
                                      ignore_waters=ignore_waters)

# capture to log
# circuitous as I needed to debug...
logger = ph.configure_logger()
logger.handlers[0].setLevel(logging.WARNING)  # logging.WARNING = 30
pyrosetta.init(extra_options=extra_options,
               #set_logging_handler=True
               )

# Uploads

In [ ]:
#@title Upload Template model
#@markdown Upload PDB file of the template model used for placing the merger.
#@markdown Suggestion: use a model bound to the native ligand or similar
#@markdown as the side chains will be poised for action!
#@markdown NB. this PDB and those of the hits need to be superposed.

#@markdown Alternatively tick here for demo values (SARS-CoV-2 MPro):

use_mpro = False #@param {type:"boolean"}

if use_mpro:
    from fragmenstein import mpro_data

    pdbblock = mpro_data.get_template()
else:
    from google.colab import files  # noqa

    uploaded = files.upload()
    assert len(uploaded) == 1, 'Upload only one template. Hits later.'
    pdbblock = list(uploaded.values())[0]
    if isinstance(pdbblock, bytes):  # dont recall what format are txt sent as...
        pdbblock = pdbblock.decode('utf8')  # noqa its bytes if it tested so

In [ ]:
#@title Upload hits or bound hits
#@markdown As the decision is based on extension, please no random extensions...
#@markdown `mol.try2.mol` is fine, but `mol.molecule` or `mol.mol.txt` are not.

#@markdown ### Option 1.
#@markdown If you have a single multientry sdf file, upload that.

#@markdown ### Option 2.
#@markdown If you have multiple mol files, upload those.
#@markdown RDKit handles mdl-mol files better than mol2 files.

#@markdown ### Option 3.
#@markdown If you require your ligand to be extracted from pdb files
#@markdown upload those and specify the three-letter code of the ligand
#@markdown (this is the 3-letter name within the provided PDB(s) not the intended name for the merger!):
ligand_residue_name = 'LIG'  #@param {type:"string"}
#@markdown This latter option is über-recommended if you have a covalently bound ligand
#@markdown but your extracted mol files are not or lack bond order.
#@markdown Note that you'll have to also upload a SMILES file (`.smi`)
#@markdown —this is just a tab separated table of SMILES tab name.

#@markdown If your PDB lacks `CONECT` entries (you monster), tick this:
proximityBonding = False  #@param {type:"boolean"}

#@markdown **Alternative** if to use demo data from covid moonshoot check use next cell.

# Go!
import os
from google.colab import files  # noqa
from rdkit import Chem
from typing import *
from warnings import warn

# ======================================
# get
uploaded = files.upload()
# sort
uploaded_split = {k: [] for k in ('smi', 'sdf', 'mol', 'mol2', 'pdb')}
for filename in uploaded:
    extension = os.path.splitext(filename)[1].lower()[1:]
    if extension not in uploaded_split:
        raise ValueError(f'The extension {extension} is not coded for')
    data = uploaded[filename]  # str or bytes??
    if isinstance(data, bytes):
        data = data.decode('utf8')
    uploaded_split[extension].append(dict(filename=filename, data=data))

# parse SMILES for PDB
if uploaded_split['smi']:
    # smilesdex? yes, I am going to hell for using not only Hungarian notation in Python, 
    # but a Pokémon flavoured one.

    smilesdex: Dict[str, str] = dict(map(lambda line: line.split('\t')[-1::-1],
                                         data.replace('\n\n', '\n').strip().split('\n'))
                                     )
else:
    smilesdex = {}


# parsers
def read_sdf(filename: str, data: str) -> List[Chem.Mol]:
    mols = []
    # I am pretty sure there's no way to run `Chem.SDMolSupplier` on a block
    with open(os.path.join(input_folder, filename), 'w') as fh:
        fh.write(data)
    with Chem.SDMolSupplier(filename, removeHs=False) as suppl:
        for mol in suppl:
            # do I need to add a name??
            mols.append(mol)
    return mols


def read_mol(filename: str, data: str, fun: Callable) -> Chem.Mol:
    mol = fun(data)
    name = os.path.splitext(filename)[0]
    mol.SetProp('_Name', name)
    return mol


def get_smiles(filename: str, smilesdex: Dict[str, str]) -> Union[None, str]:
    name = os.path.splitext(filename)[0]
    if name in smilesdex:
        return smilesdex[name]
    else:
        warn(f'Could not find matching SMILES to {name}')
        return None

from fragmenstein import Victor
# parse molecules...
hits = []
for entry in uploaded_split['sdf']:
    hits.extend(read_sdf(entry['filename'], entry['data']))
for entry in uploaded_split['mol']:
    hits.append(read_mol(entry['filename'], entry['data'], Chem.MolFromMolBlock))
for entry in uploaded_split['mol2']:
    hits.append(read_mol(entry['filename'], entry['data'], Chem.MolFromMol2Block))
for entry in uploaded_split['pdb']:
    smiles: Union[None, str] = get_smiles(entry['filename'], smilesdex)
    hits.append(Victor.extract_mol(name=entry['filename'],
                                   block=entry['data'],
                                   smiles=smiles,
                                   ligand_resn=ligand_residue_name,
                                   proximityBonding=proximityBonding,
                                   throw_on_error=True)
                )

# pollute the directory with files
# with Chem.SDWriter(os.path.join(input_folder, 'provided.sdf')) as w:
#     for hit in hits:
#         w.write(hit)

display_mols(hits)

In [ ]:
#@title Alternative source of hits: Demo data
#@markdown If using MPro template, you can use the MPro Covid moonshot data too.
n_hits = 5  #@param {type:"integer"}
#@markdown Merging Covid moonshot leads is kind of weird thing to do (cf. Lipinski rule). Max Dalton:
size_cutoff = 200  #@param {type:"integer"}

hits = mpro_data.get_n_filtered_mols(amount=n_hits, mw_cutoff=size_cutoff)
display_mols(hits)

In [ ]:
#@title Alternative source of hits: custom code?
#@markdown Uncollapse this cell for code

# some filter for either option:

def some_filter(filename:str) -> bool:
    name, extension = os.path.splitext(filename)
    return extension != '.mol' and filename.count('-') > 1

# ======== Example... local

import os

hits: List[Chem.Mol] = []
for filename in os.listdir('somefolder/molfiles/'):
    if not some_filter(filename):
        continue
    mol: Chem.Mol = Chem.MolFromMolFile( f'somefolder/molfiles/{filename}' )
    mol.SetProp('_Name', filename)
    hits.append(mol)
display_mols(hits)

# ======== Example... retrieve from GitHub
!pip install PyGithub

from github import Github
from github.ContentFile import ContentFile
import os
from typing import Tuple, List
from rdkit import Chem
# -------------------------------------------------------
def _content2mol(content:ContentFile) -> Chem.Mol:
    if content.path.count('-') > 1:
        return
    # "molfiles/diamond-x0158_B.mol"
    filename = os.path.split(content.path)[1]
    if not some_filter(filename):
        return None
    mol: Chem.Mol = Chem.MolFromMolBlock( content.decoded_content.decode('utf8') )
    mol.SetProp('_Name', filename)
    return mol

def get_repo(username:str, repo_name:str) -> Github:
    return Github().get_user(username).get_repo(repo_name)

def hits_from_github(username:str, repo_name:str, folder:str) -> List[Chem.Mol]:
    repo = get_repo(username, repo_name)
    return list(filter(lambda x: x is not None, map(_content2mol, repo.get_contents(folder))))

# ----------------- Change these:
all_hits:List[Chem.Mol] = hits_from_github(username='matteoferla',
                        repo_name='NSP3-macrodomain',
                        folder='molfiles')
hits = all_hits[:20]
pdbblock:str = get_repo(username='matteoferla', repo_name='NSP3-macrodomain').get_contents("molfiles/template.pdb").decoded_content.decode('utf8')

# Tweaks

In [ ]:
#@title Optionally prepare it (1/3)
#@markdown This will be done by 

#@markdown 1. loading it in PyRosetta
#@markdown 2. Optionally energy minimising around a target
#@markdown 3. Optionally remove some molecules

#@markdown ### Step 1
#@markdown If the model has novel ligands, they will be loaded.
#@markdown But to do this a residue type  (=topology) needs to be made or loaded.
#@markdown These are saved as "params files".
#@markdown These following options control both the "acceptor" and "donor" poses (if uploaded).
#@markdown ### Params
#@markdown * Some compounds are parameterised in the database folder of rosetta,
#@markdown others in the PDB component database (if loaded).
#@markdown * Uses the params defined in the cell of the acceptor pose.
#@markdown * If there is no topology avalaible one will be made.
#@markdown * If a params file is present in the working folder it will use it.
#@markdown * See below or visit https://params.mutanalyst.com/ to generate them (upload the with the folder icon on the left).

#@markdown This forces it (a bit silly):
force_parameterisation = False  #@param {type:"boolean"}
#@markdown If it needs to be parameterised make it protonated for pH 7?
neutralize_params = True  #@param {type:"boolean"}
save_params = True  #@param {type:"boolean"}

#@markdown If a params file is present in the working folder it will use it.
#@markdown Leave this blank... otherwise  (comma separated w/ no rando spaces):
extra_params_files_to_use = ''  #@param {type:"string"}
extra_params = [f for f in extra_params_files_to_use.split(',') if f]
use_all_folder_params = ''  #@param {type:"boolean"}
if use_all_folder_params:
    present_params = [filename for filename in os.listdir() if os.path.splitext(filename) == '.params']
else:
    present_params = []
print('loading pose...')
template_pose = ph.ligands.load.parameterized_pose_from_pdbblock(pdbblock,
                                                                 wanted_ligands=[],
                                                                 force_parameterisation=force_parameterisation,
                                                                 neutralise_params=neutralize_params,
                                                                 save_params=save_params,
                                                                 overriding_params=extra_params + present_params)

In [ ]:
#@title Optional energy minimisation around a target (2/3)
#@markdown (Requires previous cell run)
assert 'template_pose' in globals(), 'Step 1 was not run'

#@markdown If use density map is true, you will be prompted to upload a density map.
#@markdown Upload a f0fc ccp4 or a mrc map. (not a ccp4 difference map, 
#@markdown a mtz reciprocal space map or a pirate treasure map)
#@markdown The map needs to be in the same position as the template.
use_density_map = True  #@param {type:"boolean"}
#@markdown The whole structure could be minimised, but that would be pointless costly timewise
#@markdown for this task.
#@markdown Specify what residue (amino acid or ligand) to centre around 
center_residue_index = 1  #@param {type:"integer"}
center_residue_chain = 'A'  #@param {type:"string"}
center_index: int = template_pose.pdb_info().pdb2pose(res=center_residue_index, chain=center_residue_chain)
assert center_index != 0, 'That residue does not exist!'

#@markdown Specify which neighbouring residues to select in one of three ways:

#@markdown (1) Cutoff distance for the neighbouring residues (in Ångströms) (centroid to centroid)?
#@markdown set to zero to not use.
neighborhood_radius = 1  #@param {type:"integer"}
#@markdown (2) Cutoff distance for the neighbouring residues (in Ångströms) (closest atom to closest atom)?
#@markdown set to zero to not use.
cc_neighborhood_radius = 0  #@param {type:"integer"}
#@markdown (3) Max number of neighbouring residues to choose?
#@markdown set to zero to not use.
n_neighbors = 0  #@param {type:"integer"}

#@markdown ## Minimisation
#@markdown How many cycles of FastRelax to use? 3–15
cycles = 3  #@param {type:"integer"}
#@markdown to change scorefunctions and so forth edit the code.

#@markdown Note. The class Igor has two classmethods that normally 
#@markdown perform these template minimisation steps (`Igor.download_map` and `Igor.relax_with_ED`),
#@markdown but this a cruder and quicker minimisation (local).

# Get map
if use_density_map:
    map_filename = 'uploaded_map.ccp4'
    uploaded = files.upload()
    assert len(uploaded) == 1, 'wrong number of files (only one plz)'
    filename = list(uploaded.keys())[0]
    mapblock = list(uploaded.values())[0]
    with open(os.path.join(input_folder, filename), 'wb') as fh:
        fh.write(mapblock)
    # this can be done with `Igor.relax_with_ED`, but I wanted the option here to do
    # it with or without the map
    ed = ph.prep_ED(template_pose, map_filename)
    assert ed.matchPose(template_pose) > 0.5, 'This is a rubbish fit. Upload the right map.'

# prep scorefunction
scorefxn = pyrosetta.get_fa_scorefxn()
if use_density_map:
    scorefxn.set_weight(pyrosetta.rosetta.core.scoring.ScoreType.elec_dens_fast,
                        30)

selector = pyrosetta.rosetta.core.select.residue_selector
resi_sele = selector.ResidueIndexSelector(center_index)
if neighborhood_radius != 0:
    neighbor_sele = selector.NeighborhoodResidueSelector(resi_sele,
                                                         distance=neighborhood_radius,
                                                         include_focus_in_subset=True)
elif cc_neighborhood_radius != 0:
    neighbor_sele = selector.CloseContactResidueSelector()
    neighbor_sele.central_residue_group_selector(resi_sele)
    neighbor_sele.threshold(cc_neighborhood_radius)
elif n_neighbors != 0:
    neighbor_sele = selector.NumNeighborsSelector(n_neighbors, 20)
    # Ah. True. NumNeighborsSelector does not work in PyRosetta.
    raise NotImplementedError
else:
    raise ValueError

# relax
movemap = pyrosetta.MoveMap()
movemap.set_bb(allow_bb=neighbor_sele.apply(template_pose))
movemap.set_chi(allow_chi=neighbor_sele.apply(template_pose))
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, cycles)
relax.set_movemap(movemap)
relax.apply(template_pose)

pdbblock = ph.get_pdbstr(template_pose)

In [ ]:
#@title Optional remove specified stuff (3/3)
#@markdown (Requires the cell two back to be run)
import re
from warnings import warn

unwanted_residue_names_raw = 'HOH'  #@param {type:"string"}
unwanted_residue_names_raw = re.sub(r'[\W]', ' ', unwanted_residue_names_raw)
unwanted_residue_names = unwanted_residue_names_raw.split()
selector = pyrosetta.rosetta.core.select.residue_selector
for unwanted_resn in unwanted_residue_names:
    sele = selector.ResidueNameSelector()
    sele.set_residue_name3(unwanted_resn)
    try:
        sele.apply(template_pose)
    except RuntimeError:
        # ResidueNameSelector: XYZ is not a valid residue type name.
        warn(f'There is no residue {unwanted_resn}!')
    for idx in reversed(list(selector.ResidueVector(selector.apply(template_pose)))):
        template_pose.delete_residue_slow(idx)
    assert len(selector.ResidueVector(sele.apply(template_pose))) == 0

pdbblock = ph.get_pdbstr(template_pose)

# Enter Victor Fragmenstein's laboratory!

In [ ]:
#@title Merge/link -> find similars -> place similars
#@markdown Three step process:

#@markdown 1. the hits are combined pairwise
#@markdown 2. the mergers are queried in the SmallWorld server against the Enamine REAL DB
#@markdown 3. the purchasable similars are placed

#@markdown In the documentation the example uses `sqlitedict.SqliteDict`
#@markdown as this avoids dramas from segfaults from `KeyboardInterrupt` or funky entries.

# Okay, the code below contains some black magic.
# a Chem.Mol is sent down the pipe to the subprocess pickled.
# But this loses its properties (`mol.HasProp`).
# unless this dark ritual is performed:
# https://github.com/matteoferla/Fragmenstein/blob/master/documentation/mol_properties.md

#@markdown &#9888; In this notebook, ligand efficiency against the filtered set is used for ranking.
#@markdown Ranking is a topic into itself. So only simple ranking options are presented here:
ranking = '∆∆G' #@param ["LE", "∆∆G", "comRMSD"]
#@markdown ∆∆G: this has the issue that a greater number of atoms will result in a lower score,
#@markdown even if each is not contributing much.
#@markdown LE: Ligand efficiency is the most correct way to rank, but will result in similarly sized compounds to the hits, which is not desirable in fragment building.
#@markdown comRMSD: by sorting by combined RMSD the most faithful hits will be placed first.

#@markdown Angstrom distance
joining_cutoff = 5  #@param {type:"integer"}
#@markdown Angstrom distance
quick_reananimation = True  #@param {type:"boolean"}
#@markdown Convalent residue (cysteine only out of the box).
#@markdown Set as '' if noncovalent. '145A' is for MPro demo data.
covalent_resi = '145A' #@param {type:"string"}
if covalent_resi in ('', 'none', 'None', 'False', 'false', '0'):
    covalent_resi = None

# ============================================================================================
# ## Define the process

#@markdown The mergers may not be purchasable.
#@markdown As a result here the purchasable similars in Enamine Real can be sought
find_similars = True #@param {type:"boolean"}
topN_to_pick = 10  #@param {type:"integer"}
place_similars = True #@param {type:"boolean"}
#@markdown For the placement of similars use the original hits or the unminimised merger?
use_originals = False  #@param {type:"boolean"}

place_similars = find_similars and place_similars


import os, re
import pyrosetta, logging
import pandas as pd
from rdkit import Chem
from fragmenstein import Victor, Laboratory

Victor.work_path = output_folder
Victor.monster_throw_on_discard = True  # stop this merger if a fragment cannot be used.
Victor.monster_joining_cutoff = joining_cutoff  # Å
Victor.quick_reanimation = quick_reananimation  # for the impatient
Victor.error_to_catch = Exception  # stop the whole laboratory otherwise
#Victor.enable_stdout(logging.ERROR)
Victor.enable_logfile(os.path.join(output_folder, 'demo.log'), logging.ERROR)

# calculate !
lab = Laboratory(pdbblock=pdbblock, covalent_resi=covalent_resi)
n_cores = 1  #@param {type:"integer"}
combinations:pd.DataFrame = lab.combine(hits, n_cores=n_cores)


# =============================================================================================
# ## plot results


import plotly.express as px
from IPython.display import display

fig = px.histogram(combinations,
                   x='outcome',
                   category_orders={'outcome': lab.category_labels},
                   title='Distribution of Combination outcome')
fig.show()

# =============================================================================================
# ## Reverse the warhead...
# this is really unusual and janky way of doing it as one ought to know the metadata already...

from rdkit.Chem import AllChem
from fragmenstein import Victor
from typing import *
from warnings import warn

warhead_names = []
unreacted_smiles = []
for i, row in combinations.iterrows():
    unrxn, wn = Victor.guess_warhead(row.smiles) #: Tuple[str, str]
    warhead_names.append(wn)
    unreacted_smiles.append(unrxn)

combinations['unreacted_smiles'] = unreacted_smiles
combinations['warhead_type'] = warhead_names

# save

combinations.to_csv('combinations.csv')
combinations.to_pickle('combinations.p')

# =============================================================================================
# ## top 10
best_combinations = combinations.loc[combinations.outcome == 'acceptable'].sort_values(ranking).reset_index(drop=True).head(topN_to_pick)
if len(best_combinations):
    print(f'Top {topN_to_pick} mergers/linkers sorted by {ranking}')
    #PandasTools.AddMoleculeColumnToFrame(best_combinations,'smiles','molecule',includeFingerprints=False)
    display(best_combinations.drop(['unmin_binary', 'min_binary'], axis=1))
else:
    display(combinations.error)
    reasons = combinations.error.astype(str).str.split(r'^(\w+)\:', expand=True)[1].value_counts().to_dict()
    raise RuntimeError(f'The combinations failed because {reasons}')

# =============================================================================================
# ### Place purchasable similars

from smallworld_api import SmallWorld
from warnings import warn

sws = SmallWorld()
# this call requires an internet connection
chemical_databases:pd.DataFrame = sws.retrieve_databases()

if find_similars:
    similars = sws.search_many(best_combinations.unreacted_smiles,
                               dist=25,
                               db=sws.REAL_dataset,
                               tolerated_exceptions=Exception)

    similars['inspirations'] = similars.query_index.map( best_combinations.regarded.to_dict() )
    similars['merger'] = similars.query_index.map( best_combinations.smiles.to_dict() )
    similars['merger_∆∆G'] = similars.query_index.map( best_combinations['∆∆G'].to_dict() )
    similars['inspiration_mols'] = similars.query_index.map( best_combinations.hit_mols.to_dict() )
    similars['merger_unminimized_mol'] = similars.query_index.map( best_combinations.unminimized_mol.to_dict() )
    similars['merger_minimized_mol'] = similars.query_index.map( best_combinations.minimized_mol.to_dict() )
    similars.to_csv('similars.csv')
    similars.to_pickle('similars.p')

    display(similars[['smiles', 'name', 'topodist', 'inspirations', 'merger', 'merger_∆∆G']])

# ============ place the similars ==================
if place_similars:
    if use_originals:
        similars['hits'] = similars.inspiration_mols
    else:
        # make a list of one, the unminimised merger
        similars['hits'] = similars.merger_unminimized_mol.apply(lambda m: [m])

    lab = Laboratory(pdbblock=pdbblock, covalent_resi=covalent_resi)
    placements:pd.DataFrame = lab.place(similars, expand_isomers=False, n_cores=n_cores)
    display(placements)
    placements['const_ratio'] = placements['N_constrained_atoms'] / (
                placements['N_constrained_atoms'] + placements['N_unconstrained_atoms'])

    from rdkit import Chem
    from typing import *

    m = similars.drop_duplicates('name').set_index('name').to_dict()
    placements['merger_∆∆G'] = placements['name'].map(m['merger_∆∆G'])
    placements['merger_minimized_mol'] = placements['name'].map(m['merger_minimized_mol'])
    placements['merger_unminimized_mol'] = placements['name'].map(m['merger_unminimized_mol'])
    placements.rename(columns={'unminimized_mol': 'enamine_unminimized_mol',
                               'minimized_mol': 'enamine_minimized_mol'}, inplace=True)
    placements['merger_inspiration_names'] = placements['name'].map(m['inspirations'])
    placements['merger_inspiration_mols'] = placements.hit_mols
    nan_to_list = lambda value: value if isinstance(value, list) else []
    placements['disregarded'] = placements.disregarded.apply(nan_to_list)
    placements['regarded'] = placements.regarded.apply(nan_to_list)

    placements.to_csv('placements.csv')
    placements.to_pickle('placements.p')

    # NB: more than 2 in 3 constrained is actually uncommon with enamine real for larger mergers.
    # hence the 1 in 2

    best_placements = placements.loc[
        (placements.outcome == 'acceptable')
        & (placements.const_ratio > 1/2) ].sort_values(ranking).reset_index(drop=True).head(topN_to_pick)
    if len(best_placements):
        print(f'Top {topN_to_pick} placements sorted by {ranking}')
        #PandasTools.AddMoleculeColumnToFrame(best_combinations,'smiles','molecule',includeFingerprints=False)
        # noisy_fields = ['hit_mols', 'merger_unminimized_mol',
        #                           'merger_unminimized_mol',
        #                           'unmin_binary', 'min_binary']
        noisy_fields = []
        display( best_placements.drop(noisy_fields, axis=1) )
    else:
        display(placements.error)
        reasons = placements.error.astype(str).str.split(r'^(\w+)\:', expand=True)[1].value_counts().to_dict()
        raise RuntimeError(f'The placements failed because {reasons}')

#PandasTools.AddMoleculeColumnToFrame(best_placements,'smiles','molecule',includeFingerprints=False)


# =============================================================================================
# ### Results redux

from IPython.display import clear_output, HTML, display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

headerify: Callable[[str], HTML] = lambda header: HTML(f'<h3>{header}</h3>')

clear_output()
if place_similars:
    fig = px.histogram(placements,
                       x='outcome',
                       category_orders={'outcome': lab.category_labels},
                       title='Distribution of Placement outcome')
    fig.show()


display(headerify('Provided hits'))
display_mols(hits)
display(headerify('Step 1. Combine'))
fig.show()
print(f'Top {topN_to_pick} mergers/linkers sorted by {ranking}')
display(best_combinations.drop(['unmin_binary', 'min_binary'], axis=1))
display_mols(best_combinations.unminimized_mol)

def show3D_combined(index_to_show):
    row = best_placements.iloc[index_to_show]
    print(f'Green: hits {row["merger_inspiration_names"]}')
    print('Cyan: merger')
    display(make_3Dview(pdbblock, {'greenCarbon': row.hit_mols,
                           'cyanCarbon': [row.minimized_mol]}))

print('In the sorted combinations table (`best_combinations`) which index do you want to see:')
scale = widgets.IntSlider(min=0,max=len(best_combinations)-1, step=1, value=0)
interact(show3D_combined, index_to_show=scale)
#display(similars)
if place_similars:

    def show3D_placed(index_to_show):
        row = best_placements.iloc[index_to_show]
        print(f'Green: hits {row["merger_inspiration_names"]}')
        print('Cyan: merger')
        print(f'Magenta: Enamine Real purchasable {row["name"]}')
        display(make_3Dview(pdbblock, {'greenCarbon': row.merger_inspiration_mols,
                               'cyanCarbon': [row.merger_minimized_mol],
                               'magentaCarbon': [row.enamine_minimized_mol]}))

    display(headerify('Step 2. Placement of purchasable similars'))
    display(headerify(f'Top {topN_to_pick} Placements'))
    display( best_placements.drop(noisy_fields, axis=1) )

    print('In the sorted combinations table (`best_combinations`) which index do you want to see:')
    scale = widgets.IntSlider(min=0,max=len(best_placements)-1, step=1, value=0)
    interact(show3D_placed, index_to_show=scale)
elif find_similars:
    display(headerify('Purchasable similars'))
    display(similars)